In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
import keras

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Flatten,AveragePooling2D,Dense,Dropout
from tensorflow.keras.models import Model

data='D:\Destination\dataset'
category=["New_Front","New_Rear","Old_Front","Old_Rear"]
#category2=['video_data']
IMG_SIZE=227
test=[]
train = []
def create_training_data(t,d,c):
    for cat in c:
        path = os.path.join(d, cat)
        class_num = c.index(cat)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
                t.append([new_array, class_num])
            except Exception as e:
                pass
    return t
train=create_training_data(train,data,category)
#test=create_training_data(test,data,category2)

def process_images(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (227,227))
    return image, label


random.shuffle(train)
x=[]
y=[]
for feat, label in train:
    x.append(feat)
    y.append(label)
x=np.array(x).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y=np.array(y)
x=x/255.0
validation_images, validation_labels = x[:285], y[:285]
train_images, train_labels = x[285:], y[285:]
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))
train_ds = (train_ds.map(process_images).batch(batch_size=32, drop_remainder=True))
validation_ds = (validation_ds.map(process_images).batch(batch_size=32, drop_remainder=True))

"""
x_test=[]
y_test=[]
for feat, label in test:
    x_test.append(feat)
    y_test.append(label)
x_test=np.array(x_test).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_test=np.array(y_test)
x_test=x_test/255.0
"""

baseModel = MobileNetV2(input_shape=[227,227,3], weights="imagenet",include_top=False)
x=baseModel.output
x=AveragePooling2D(pool_size=(7,7))(x)
x= Flatten(name="flatten")(x)
x= Dense(4096, activation='relu')(x)
x= Dropout(0.5)(x)
x= Dense(4096, activation='sigmoid')(x)
x= Dropout(0.5)(x)
x= Dense(4, activation='softmax')(x)
model = Model(inputs=baseModel.input,outputs=x)

model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit(train_ds,epochs=10,validation_data=validation_ds)

#results = model.evaluate(x_test, y_test, batch_size=32)
#print("test loss, test acc:", results)

#prediction = model.predict(x_test[:1])
#print("prediction shape:", prediction.shape)


# Saving and loading model and weights
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
model_all_json = model.to_json()
with open("model_all.json", "w") as json_file:
    json_file.write(model_all_json)
# serialize weights to HDF5
model.save_weights("model_all.h5")
print("Saved model to disk")

# load json and create model
json_file = open('model_all.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_all.h5")
print("Loaded model from disk")

model.save('model_all.hdf5')
loaded_model=load_model('model_all.hdf5')


